<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# Fourier-based Option Pricing

For several reasons, it is beneficial to have available alternative valuation and pricing approaches to the Monte Carlo simulation approach. One application area is to **benchmark Monte Carlo-based valuation results** against other (potentially more accurate) results. Another area is **model calibration to liquidly traded vanilla instruments** where generally faster numerial methods can be applied.

This part introduces **Fouried-based valuation functions** and benchmarks valuation results from the "standard", simulation-based DX Analytics modeling approach to output of those functions. 

In [1]:
import dx
import datetime as dt

## Risk Factors

The examples and benchmarks to follow rely on four different models:

* geometric Brownian motion (Black-Scholes-Merton 1973)
* jump diffusion (Merton 1976)
* stochastic volatility (Heston 1993)
* stochastic volatility jump diffusion (Bates 1996)

For details on these models and the Fourier-based option pricing approach refer to Hilpisch (2015) (cf. http://eu.wiley.com/WileyCDA/WileyTitle/productCd-1119037999.html).

We first define the single **market and valuation environments**.

In [2]:
# constant short rate
r = dx.constant_short_rate('r', 0.01)

In [3]:
# geometric Brownian motion
me = dx.market_environment('me', dt.datetime(2015, 1, 1))
me.add_constant('initial_value', 100.)
me.add_constant('volatility', 0.2)
me.add_constant('final_date', dt.datetime(2015, 12, 31))
me.add_constant('currency', 'EUR')

In [4]:
# jump component
me.add_constant('lambda', 0.4)
me.add_constant('mu', -0.6)
me.add_constant('delta', 0.2)

In [5]:
# stochastic volatiltiy component
me.add_constant('rho', -.5)
me.add_constant('kappa', 5.0)
me.add_constant('theta', 0.02)
me.add_constant('vol_vol', 0.3)

In [6]:
# valuation environment
val_env = dx.market_environment('val_env', dt.datetime(2015, 1, 1))
val_env.add_constant('paths', 55000)
    # 25,000 paths
val_env.add_constant('frequency', 'D')
    # weekly frequency
val_env.add_curve('discount_curve', r)
val_env.add_constant('starting_date', dt.datetime(2015, 1, 1))
val_env.add_constant('final_date', dt.datetime(2015, 12, 31))

In [7]:
# add valuation environment to market environment
me.add_environment(val_env)

Equipped with the single market environments and the valuation environment, we can instantiate the **simulation model objects**.

In [8]:
gbm = dx.geometric_brownian_motion('gbm', me)

In [9]:
jd = dx.jump_diffusion('jd', me)

In [10]:
sv = dx.stochastic_volatility('sv', me)

In [11]:
svjd = dx.stoch_vol_jump_diffusion('svjd', me)

## Plain Vanilla Put and Call Options

Based on the just defined risk factors, we define 8 diffent options---a **European put and call option per risk factor**, respectively.

In [12]:
# market environment for the options
me_option = dx.market_environment('option', dt.datetime(2015, 1, 1))
me_option.add_constant('maturity', dt.datetime(2015, 12, 31))
me_option.add_constant('strike', 100.)
me_option.add_constant('currency', 'EUR')
me_option.add_environment(me)
me_option.add_environment(val_env)

In [13]:
euro_put_gbm = dx.valuation_mcs_european_single('euro_put', gbm, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_gbm = dx.valuation_mcs_european_single('euro_call', gbm, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [14]:
euro_put_jd = dx.valuation_mcs_european_single('euro_put', jd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_jd = dx.valuation_mcs_european_single('euro_call', jd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [15]:
euro_put_sv = dx.valuation_mcs_european_single('euro_put', sv, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_sv = dx.valuation_mcs_european_single('euro_call', sv, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [16]:
euro_put_svjd = dx.valuation_mcs_european_single('euro_put', svjd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_svjd = dx.valuation_mcs_european_single('euro_call', svjd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

## Valuation Benchmarking

In this sub-section, we benchmark the **Monte Carlo value estimates** against the **Fourier-based pricing results**.

In [17]:
import numpy as np
import pandas as pd

We first define some parameters used throughout.

In [18]:
freq = '2m'  # used for maturity definitions
periods = 3  # number of intervals for maturity grid
strikes = 5  # number of strikes per maturity
initial_value = 100  # initial value for all risk factors
start = 0.8  # lowest strike in percent of spot
end = 1.2  # highest strike in percent of spot
start_date = '2015/3/1'  # start date for simulation/pricing

### Geometric Brownian Motion

We need to initialize the valuation object first.

In [19]:
euro_put_gbm.present_value()
  # method call needed for initialization

7.448555

There is a **valuation class for European put and call options in the Black-Scholes-Merton model** available called `BSM_european_option`. It is based on the analytical pricing formula for that model and is instantiated as follows:

In [20]:
bsm_option = dx.BSM_european_option('bsm_opt', me_option)

The following routine benchmarks the Monte Carlo value estimates for the **European put option** against the output from the valuation object based on the analytical pricing formula. The results are quite good since this model is quite easy to discretize exactly and therefore generally shows good convergence of the Monte Carlo estimates.

In [21]:
%%time
# European put
print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    bsm_option.maturity = maturity
    euro_put_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_put_gbm.update(strike=strike)
        mcs = euro_put_gbm.present_value()
        bsm_option.strike = strike
        ana = bsm_option.put_value()
        print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f '
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100))

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0336 |  0.0338 | -0.0002 |   -0.56 
0.244 |  90.000 |  0.6445 |  0.6524 | -0.0079 |   -1.22 
0.244 | 100.000 |  3.7901 |  3.8130 | -0.0229 |   -0.60 
0.244 | 110.000 | 10.6209 | 10.6957 | -0.0748 |   -0.70 
0.244 | 120.000 | 19.7029 | 19.8537 | -0.1508 |   -0.76 
0.411 |  80.000 |  0.1645 |  0.1748 | -0.0102 |   -5.85 
0.411 |  90.000 |  1.3395 |  1.3241 |  0.0154 |    1.16 
0.411 | 100.000 |  4.8414 |  4.8985 | -0.0571 |   -1.17 
0.411 | 110.000 | 11.3837 | 11.4275 | -0.0439 |   -0.38 
0.411 | 120.000 | 19.9163 | 20.0325 | -0.1162 |   -0.58 
0.578 |  80.000 |  0.3718 |  0.3917 | -0.0200 |   -5.10 
0.578 |  90.000 |  1.9340 |  1.9466 | -0.0126 |   -0.65 
0.578 | 100.000 |  5.7426 |  5.7593 | -0.0167 |   -0.29 
0.578 | 110.000 | 12.0548 | 12.0934 | -0.0386 |   -0.32 
0.578 | 120.000 | 20.2263 | 20.3153 | -0.0890 |   -0.44 
CPU times: user 13.3 s, sys: 4.98 s, total: 18.2 s
Wall time: 18.3 s


The same now for the **European call option**.

In [22]:
euro_call_gbm.present_value()
  # method call needed for initialization

8.463904

In [23]:
%%time
# European calls
print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    euro_call_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_call_gbm.update(strike=strike)
        mcs = euro_call_gbm.present_value()
        bsm_option.strike = strike
        bsm_option.maturity = maturity
        ana = bsm_option.call_value()
        print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f ' \
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100))

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0804 | 20.2286 | -0.1483 |   -0.73 
0.244 |  90.000 | 10.8153 | 10.8716 | -0.0563 |   -0.52 
0.244 | 100.000 |  4.0216 |  4.0565 | -0.0349 |   -0.86 
0.244 | 110.000 |  0.9532 |  0.9636 | -0.0104 |   -1.08 
0.244 | 120.000 |  0.1489 |  0.1460 |  0.0030 |    2.03 
0.411 |  80.000 | 20.3853 | 20.5029 | -0.1176 |   -0.57 
0.411 |  90.000 | 11.6377 | 11.6932 | -0.0555 |   -0.47 
0.411 | 100.000 |  5.2632 |  5.3086 | -0.0455 |   -0.86 
0.411 | 110.000 |  1.8397 |  1.8787 | -0.0390 |   -2.07 
0.411 | 120.000 |  0.5155 |  0.5246 | -0.0091 |   -1.74 
0.578 |  80.000 | 20.7878 | 20.8528 | -0.0651 |   -0.31 
0.578 |  90.000 | 12.4321 | 12.4654 | -0.0333 |   -0.27 
0.578 | 100.000 |  6.3127 |  6.3357 | -0.0230 |   -0.36 
0.578 | 110.000 |  2.7123 |  2.7274 | -0.0151 |   -0.56 
0.578 | 120.000 |  0.9930 |  1.0070 | -0.0140 |   -1.39 
CPU times: user 13 s, sys: 4.82 s, total: 17.8 s
Wall time: 17.8 s


### Benchmarking Function

All other valuation benchmarks are generated with **Fourier-based pricing functions** for which the handling is identical. We therefore use the following function for the benchmarks from now on:

In [24]:
def valuation_benchmarking(valuation_object, fourier_function):
    print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
    for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
        valuation_object.update(maturity=maturity)
        me_option.add_constant('maturity', maturity)
        for strike in np.linspace(start, end, strikes) * initial_value:
            T = (maturity - me_option.pricing_date).days / 365.
            valuation_object.update(strike=strike)
            mcs = valuation_object.present_value()
            me_option.add_constant('strike', strike)
            fou = fourier_function(me_option)
            print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.3f '
                % (T, strike, mcs, fou, mcs - fou, (mcs - fou) / fou * 100))

### Jump Diffusion

The next model is the jump diffusion as proposed by **Merton (1976)**.

In [25]:
euro_put_jd.present_value()
  # method call needed for initialization

14.044053

There is a Fourier-based pricing function available which is called `M76_put_value` and which is used for the benchmarking for the **European put options** that follows.

In [26]:
%time valuation_benchmarking(euro_put_jd, dx.M76_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1743 |  2.1594 |  0.0149 |   0.690 
0.244 |  90.000 |  3.2301 |  3.2826 | -0.0526 |  -1.601 
0.244 | 100.000 |  5.8903 |  5.8842 |  0.0061 |   0.103 
0.244 | 110.000 | 11.5439 | 11.6115 | -0.0676 |  -0.582 
0.244 | 120.000 | 19.9178 | 20.0857 | -0.1680 |  -0.836 
0.411 |  80.000 |  3.3626 |  3.4505 | -0.0879 |  -2.548 
0.411 |  90.000 |  5.2550 |  5.2162 |  0.0388 |   0.744 
0.411 | 100.000 |  8.0292 |  8.2266 | -0.1974 |  -2.400 
0.411 | 110.000 | 13.4297 | 13.4430 | -0.0132 |  -0.099 
0.411 | 120.000 | 20.7677 | 20.9238 | -0.1561 |  -0.746 
0.578 |  80.000 |  4.5110 |  4.6090 | -0.0980 |  -2.126 
0.578 |  90.000 |  6.7923 |  6.8782 | -0.0859 |  -1.249 
0.578 | 100.000 | 10.1470 | 10.2077 | -0.0606 |  -0.594 
0.578 | 110.000 | 15.2022 | 15.2251 | -0.0229 |  -0.151 
0.578 | 120.000 | 22.0274 | 22.0886 | -0.0612 |  -0.277 
CPU times: user 31.4 s, sys: 8.51 s, total: 39.9 s
Wall time: 39.9 s


Accordingly, the benchmarking for the **European call options** based on the Fourier-based `M76_call_value` function.

In [27]:
euro_call_jd.present_value()
  # method call needed for initialization

14.986366

In [28]:
%time valuation_benchmarking(euro_call_jd, dx.M76_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.1806 | 22.3543 | -0.1737 |  -0.777 
0.244 |  90.000 | 13.3860 | 13.5018 | -0.1159 |  -0.858 
0.244 | 100.000 |  6.0676 |  6.1277 | -0.0602 |  -0.982 
0.244 | 110.000 |  1.8523 |  1.8794 | -0.0271 |  -1.441 
0.244 | 120.000 |  0.3777 |  0.3780 | -0.0003 |  -0.074 
0.411 |  80.000 | 23.6261 | 23.7786 | -0.1525 |  -0.642 
0.411 |  90.000 | 15.5259 | 15.5853 | -0.0594 |  -0.381 
0.411 | 100.000 |  8.6186 |  8.6367 | -0.0181 |  -0.209 
0.411 | 110.000 |  3.8612 |  3.8941 | -0.0329 |  -0.844 
0.411 | 120.000 |  1.3962 |  1.4160 | -0.0198 |  -1.396 
0.578 |  80.000 | 25.0199 | 25.0701 | -0.0502 |  -0.200 
0.578 |  90.000 | 17.2504 | 17.3970 | -0.1466 |  -0.842 
0.578 | 100.000 | 10.7401 | 10.7841 | -0.0440 |  -0.408 
0.578 | 110.000 |  5.8569 |  5.8591 | -0.0022 |  -0.038 
0.578 | 120.000 |  2.7726 |  2.7803 | -0.0077 |  -0.277 
CPU times: user 31.9 s, sys: 8.89 s, total: 40.8 s
Wall time: 40.9 s


### Stochastic Volatility

Stochastic volatility models like the one of **Heston (1993)** are popular to reproduce implied volatility smiles observed in markets. First, the benchmarking for the **European put options** using the Fourier-based `H93_put_value` function.

In [29]:
euro_put_sv.present_value()
  # method call needed for initialization

5.288932

In [30]:
%time valuation_benchmarking(euro_put_sv, dx.H93_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0460 |  0.0504 | -0.0044 |  -8.772 
0.244 |  90.000 |  0.5470 |  0.5741 | -0.0271 |  -4.726 
0.244 | 100.000 |  3.2500 |  3.3204 | -0.0704 |  -2.119 
0.244 | 110.000 | 10.1319 | 10.2404 | -0.1086 |  -1.060 
0.244 | 120.000 | 19.5825 | 19.7354 | -0.1528 |  -0.774 
0.411 |  80.000 |  0.1402 |  0.1603 | -0.0201 | -12.528 
0.411 |  90.000 |  0.9406 |  1.0063 | -0.0657 |  -6.527 
0.411 | 100.000 |  3.8904 |  4.0234 | -0.1330 |  -3.305 
0.411 | 110.000 | 10.4330 | 10.5487 | -0.1158 |  -1.097 
0.411 | 120.000 | 19.5061 | 19.6293 | -0.1232 |  -0.628 
0.578 |  80.000 |  0.2612 |  0.2838 | -0.0226 |  -7.974 
0.578 |  90.000 |  1.2803 |  1.3631 | -0.0827 |  -6.069 
0.578 | 100.000 |  4.3977 |  4.5467 | -0.1490 |  -3.277 
0.578 | 110.000 | 10.6870 | 10.8389 | -0.1519 |  -1.401 
0.578 | 120.000 | 19.4572 | 19.5767 | -0.1195 |  -0.610 
CPU times: user 34 s, sys: 12.1 s, total: 46.2 s
Wall time: 45.1 s


Second, the benchmarking for the **European call options** based on the Fourier-based `H93_call_value` function.

In [31]:
euro_call_sv.present_value()
  # method call needed for initialization

6.272266

In [32]:
%time valuation_benchmarking(euro_call_sv, dx.H93_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0866 | 20.2453 | -0.1587 |  -0.784 
0.244 |  90.000 | 10.7026 | 10.7933 | -0.0907 |  -0.840 
0.244 | 100.000 |  3.4880 |  3.5639 | -0.0759 |  -2.131 
0.244 | 110.000 |  0.4715 |  0.5083 | -0.0368 |  -7.244 
0.244 | 120.000 |  0.0253 |  0.0276 | -0.0023 |  -8.220 
0.411 |  80.000 | 20.3567 | 20.4884 | -0.1317 |  -0.643 
0.411 |  90.000 | 11.2539 | 11.3754 | -0.1214 |  -1.068 
0.411 | 100.000 |  4.3375 |  4.4335 | -0.0960 |  -2.165 
0.411 | 110.000 |  0.9482 |  0.9998 | -0.0516 |  -5.162 
0.411 | 120.000 |  0.1047 |  0.1214 | -0.0167 | -13.756 
0.578 |  80.000 | 20.6250 | 20.7450 | -0.1200 |  -0.578 
0.578 |  90.000 | 11.7512 | 11.8818 | -0.1306 |  -1.099 
0.578 | 100.000 |  4.9554 |  5.1231 | -0.1677 |  -3.273 
0.578 | 110.000 |  1.3861 |  1.4729 | -0.0868 |  -5.894 
0.578 | 120.000 |  0.2257 |  0.2684 | -0.0427 | -15.918 
CPU times: user 33.4 s, sys: 11.7 s, total: 45.1 s
Wall time: 44 s


### Stochastic Volatility Jump-Diffusion

Finally, we consider the combination of the stochastic volatility and jump diffusion models from before as proposed by **Bates (1996)**. The Fourier-based pricing function for **European put options** is called `B96_put_value`.

In [33]:
euro_put_svjd.present_value()
  # method call needed for initialization

12.916162

In [34]:
%time valuation_benchmarking(euro_put_svjd, dx.B96_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1001 |  2.1638 | -0.0637 |  -2.945 
0.244 |  90.000 |  3.1980 |  3.2761 | -0.0781 |  -2.385 
0.244 | 100.000 |  5.4574 |  5.5889 | -0.1315 |  -2.354 
0.244 | 110.000 | 10.9153 | 11.0733 | -0.1580 |  -1.427 
0.244 | 120.000 | 19.7644 | 19.8344 | -0.0700 |  -0.353 
0.411 |  80.000 |  3.3605 |  3.4366 | -0.0760 |  -2.212 
0.411 |  90.000 |  5.0351 |  5.1339 | -0.0988 |  -1.925 
0.411 | 100.000 |  7.6348 |  7.7747 | -0.1399 |  -1.799 
0.411 | 110.000 | 12.4060 | 12.5744 | -0.1684 |  -1.339 
0.411 | 120.000 | 20.0165 | 20.1827 | -0.1662 |  -0.824 
0.578 |  80.000 |  4.3430 |  4.5480 | -0.2051 |  -4.509 
0.578 |  90.000 |  6.4593 |  6.7163 | -0.2570 |  -3.827 
0.578 | 100.000 |  9.3161 |  9.6585 | -0.3423 |  -3.544 
0.578 | 110.000 | 13.9156 | 14.1776 | -0.2620 |  -1.848 
0.578 | 120.000 | 20.7501 | 20.9423 | -0.1922 |  -0.918 
CPU times: user 53.6 s, sys: 16 s, total: 1min 9s
Wall time: 1min 8s


The Fourier-based counterpart function for **European call options** is called `B96_call_value`.

In [35]:
euro_call_svjd.present_value()
  # method call needed for initialization

13.894752

In [36]:
%time valuation_benchmarking(euro_call_svjd, dx.B96_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.1408 | 22.3587 | -0.2179 |  -0.975 
0.244 |  90.000 | 13.3373 | 13.4953 | -0.1580 |  -1.171 
0.244 | 100.000 |  5.7334 |  5.8325 | -0.0991 |  -1.699 
0.244 | 110.000 |  1.3008 |  1.3411 | -0.0403 |  -3.007 
0.244 | 120.000 |  0.1177 |  0.1266 | -0.0089 |  -7.021 
0.411 |  80.000 | 23.5763 | 23.7647 | -0.1884 |  -0.793 
0.411 |  90.000 | 15.2775 | 15.5030 | -0.2255 |  -1.454 
0.411 | 100.000 |  8.0391 |  8.1848 | -0.1457 |  -1.780 
0.411 | 110.000 |  2.9136 |  3.0255 | -0.1119 |  -3.699 
0.411 | 120.000 |  0.6111 |  0.6749 | -0.0637 |  -9.440 
0.578 |  80.000 | 24.6845 | 25.0091 | -0.3246 |  -1.298 
0.578 |  90.000 | 16.9131 | 17.2351 | -0.3220 |  -1.868 
0.578 | 100.000 |  9.9853 | 10.2349 | -0.2496 |  -2.439 
0.578 | 110.000 |  4.5961 |  4.8117 | -0.2156 |  -4.481 
0.578 | 120.000 |  1.4735 |  1.6340 | -0.1605 |  -9.822 
CPU times: user 52.7 s, sys: 15.5 s, total: 1min 8s
Wall time: 1min 7s


## Sources of Errors

Numerical methods like Monte Carlo simulation might suffer from different **sources of errors**, like for example:

* **discretization error**: every **discretization of a continuous time interval**---or a continuous state space to this end---leads to a so-called discretization error
* **approximation errors**: DX Analytics uses in several places approximative, **Euler-based discretization schemes** (e.g. for performance reasons and to allow for consistent correlation modeling) which are known to be biased
* **numerical errors**: the approximation of a continuous probability distribution by a **finite, discrete set of (pseudo-) random numbers** introduces also errors

**Copyright, License & Disclaimer**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

DX Analytics (the "dx library") is licensed under the GNU Affero General Public License
version 3 or later (see [http://www.gnu.org/licenses/](http://www.gnu.org/licenses/)).

DX Analytics comes with no representations
or warranties, to the extent permitted by applicable law.


<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

[http://tpq.io](http://tpq.io) | [team@tpq.io](mailto:team@tpq.io) | [http://twitter.com/dyjh](http://twitter.com/dyjh)

**Quant Platform** |
[http://quant-platform.com](http://quant-platform.com)

**Derivatives Analytics with Python (Wiley Finance)** |
[http://derivatives-analytics-with-python.com](http://derivatives-analytics-with-python.com)

**Python for Finance (O'Reilly)** |
[http://python-for-finance.com](http://python-for-finance.com)